In [18]:
import pandas as pd
import joblib
from weather_utils import get_weather_data

In [16]:
pipeline = joblib.load("yield_prediction_model.pkl")


In [20]:
def get_crop_recommendations(district: str, year: int, area_ha: float = 1.0, top_n: int = 3):
   
    
    try:
        pipeline = joblib.load("yield_prediction_model.pkl")
    except FileNotFoundError:
        print("Error: 'yield_prediction_model.pkl' not found. Make sure the model file is in the same directory.")
        return None

    
    CROP_DATA = {
        "Rice": {"msp_per_quintal": 2300, "water_req_mm": 1300},
        "Wheat": {"msp_per_quintal": 2275, "water_req_mm": 550},
        "Maize": {"msp_per_quintal": 2225, "water_req_mm": 650},
        "Sugarcane": {"msp_per_quintal": 340, "water_req_mm": 2000},
        "Cotton": {"msp_per_quintal": 7121, "water_req_mm": 800},
        "Soyabean": {"msp_per_quintal": 4892, "water_req_mm": 600},
        "Pigeonpea": {"msp_per_quintal": 7550, "water_req_mm": 450}, 
        "Chickpea": {"msp_per_quintal": 5440, "water_req_mm": 400}, 
        "Rapeseed & Mustard": {"msp_per_quintal": 5650, "water_req_mm": 350},
        "Groundnut": {"msp_per_quintal": 6783, "water_req_mm": 600},
        "Sunflower": {"msp_per_quintal": 7280, "water_req_mm": 650},
        "Barley": {"msp_per_quintal": 1850, "water_req_mm": 350},
        "Pearl Millet": {"msp_per_quintal": 2625, "water_req_mm": 300}, 
        "Kharif Sorghum": {"msp_per_quintal": 3371, "water_req_mm": 500}, 
    }
    print(f"Fetching weather forecast for {district}, {year}...")
    weather_data = get_weather_data(district, year)
    if not weather_data:
     print("Could not fetch weather data. Please check your district name and API key.")
     return None
    
    available_rainfall = weather_data.get("rainfall", 0)
    print(f"Forecasted rainfall: {available_rainfall} mm")  
    all_recommendations = []
    for crop, data in CROP_DATA.items():
        
        features = {
            "Year": year,
            "Area (1000 ha)": area_ha / 1000,
            "temp_avg": weather_data["temp_avg"],
            "temp_max": weather_data["temp_max"],
            "temp_min": weather_data["temp_min"],
            "humidity": weather_data["humidity"],
            "rainfall": available_rainfall,
            "Dist Name": district.lower(),
            "Crop": crop.upper()
        }
        features_df = pd.DataFrame([features])
        
        predicted_yield_kg_ha = pipeline.predict(features_df)[0]

        
        msp_per_quintal = data["msp_per_quintal"]
        estimated_revenue_ha = (predicted_yield_kg_ha / 100) * msp_per_quintal

        
        water_req_mm = data["water_req_mm"]
        irrigation_needed_mm = max(0, water_req_mm - available_rainfall)

        all_recommendations.append({
            "Crop": crop,
            "Predicted Yield (kg/ha)": round(predicted_yield_kg_ha, 2),
            "Estimated Revenue (₹/ha)": round(estimated_revenue_ha, 2),
            "Irrigation Needed (mm)": round(irrigation_needed_mm, 2)
        })

    sorted_recommendations = sorted(all_recommendations, key=lambda x: x["Estimated Revenue (₹/ha)"], reverse=True)
    
    top_recommendations_df = pd.DataFrame(sorted_recommendations).head(top_n)
    
    return top_recommendations_df



In [21]:
if __name__ == "__main__":
    # === INPUTS ===
    TARGET_DISTRICT = "Lucknow"
    TARGET_YEAR = 2025 # Using a future year for forecasting
    
    print("="*50)
    print("🌾 Crop Recommendation System 🌾")
    print("="*50)

    # Get the recommendations
    recommendations = get_crop_recommendations(district=TARGET_DISTRICT, year=TARGET_YEAR)

    # Display the results
    if recommendations is not None and not recommendations.empty:
        print(f"\n✅ Top 3 Crop Recommendations for {TARGET_DISTRICT} in {TARGET_YEAR} (Ranked by Revenue):\n")
        
        for index, row in recommendations.iterrows():
            print(f"--- Rank #{index + 1}: {row['Crop']} ---")
            print(f"  💰 Estimated Revenue: ₹{row['Estimated Revenue (₹/ha)']:,} per hectare")
            print(f"  🌾 Predicted Yield: {row['Predicted Yield (kg/ha)']} kg/ha")
            print(f"  💧 Irrigation Needed: Approx. {row['Irrigation Needed (mm)']} mm of additional water required.")
            print("-" * (20 + len(row['Crop'])))
    else:
        print("\nCould not generate recommendations.")

🌾 Crop Recommendation System 🌾
Fetching weather forecast for Lucknow, 2025...
Forecasted rainfall: 0.0 mm

✅ Top 3 Crop Recommendations for Lucknow in 2025 (Ranked by Revenue):

--- Rank #1: Rapeseed & Mustard ---
  💰 Estimated Revenue: ₹155,618.953125 per hectare
  🌾 Predicted Yield: 2754.320068359375 kg/ha
  💧 Irrigation Needed: Approx. 350.0 mm of additional water required.
--------------------------------------
--- Rank #2: Barley ---
  💰 Estimated Revenue: ₹50,954.87890625 per hectare
  🌾 Predicted Yield: 2754.320068359375 kg/ha
  💧 Irrigation Needed: Approx. 350.0 mm of additional water required.
--------------------------
--- Rank #3: Sugarcane ---
  💰 Estimated Revenue: ₹20,864.400390625 per hectare
  🌾 Predicted Yield: 6136.58984375 kg/ha
  💧 Irrigation Needed: Approx. 2000.0 mm of additional water required.
-----------------------------
